In [33]:
# Сменим рабочую дирректорию для удобства
import os
os.chdir('/home/slawa/HDD/my_scripts/my_otus/ML_advanced/lesson_9/hw')

In [34]:
# библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import f1_score, make_scorer
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
import xgboost

In [35]:
# загружаем данные
raw_data_file_name = 'data/raw_data/heart.csv'
raw_data = pd.read_csv(raw_data_file_name, delimiter=',', decimal='.')
raw_data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [36]:
# удаляем дубликаты
raw_data.drop_duplicates(inplace=True)

In [37]:
# удаляем мусорные данные
raw_data.drop(raw_data.loc[raw_data['thal'] == 0].index, inplace=True)

In [38]:
# считаем баланс классов
for tg in raw_data['target'].unique():
    count = raw_data.loc[raw_data['target'] == tg].shape[0]
    count_percent = count / raw_data.shape[0] * 100
    print(f'class {tg}: count {count}, percent {count_percent:.2f}%')

class 0: count 137, percent 45.67%
class 1: count 163, percent 54.33%


In [39]:
# прячем часть данных для финальной проверки
data_train, data_test = train_test_split(raw_data, test_size=0.1, stratify=raw_data['target'])
data_test.to_csv('data/test_data/final_test_data.csv', index=False)

In [40]:
# бьем оставшуюся выборку
df_train, df_test = train_test_split(data_train, test_size=0.2, stratify=data_train['target'])

In [41]:
# считаем баланс классов в оставшемся датасете
for tg in raw_data['target'].unique():
    count = raw_data.loc[raw_data['target'] == tg].shape[0]
    count_percent = count / raw_data.shape[0] * 100
    print(f'class {tg}: count {count}, percent {count_percent:.2f}%')

class 0: count 137, percent 45.67%
class 1: count 163, percent 54.33%


In [42]:
# разобьем поля по их типам
target_col = ['target']
categorical_col = ['cp', 'restecg', 'slope']
numeric_col = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'ca', 'thal']
binary_col = ['sex', 'fbs', 'exang']
all_features = categorical_col + numeric_col + binary_col

In [43]:
# собираем данные для обучения
X_train_df = df_train[all_features]
Y_train_df = df_train[target_col]
X_test_df = df_test[all_features]
Y_test_df = df_test[target_col]
print(f'train: X{X_train_df.shape}, Y{Y_train_df.shape}')
print(f'test: X{X_test_df.shape}, Y{Y_test_df.shape}')

train: X(216, 13), Y(216, 1)
test: X(54, 13), Y(54, 1)


In [44]:
# собираем конвейер предобработки данных
data_transformer = ColumnTransformer(
    [
        ("onehot", OneHotEncoder(handle_unknown="ignore"), categorical_col),
        ('scaler', StandardScaler(), numeric_col),
        ("as_is", "passthrough", binary_col)
    ]
)
preprocessor = Pipeline(steps=[("data_transformer", data_transformer)])

In [45]:
# собираем и обучаем случайный лес
classifier_pipline = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier())
    ]
)
classifier_pipline
classifier_pipline.fit(X_train_df, Y_train_df)

/home/slawa/HDD/my_scripts/my_otus/ML_advanced/venv/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('data_transformer',
                                  ColumnTransformer(transformers=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'),
                                                                   ['cp',
                                                                    'restecg',
                                                                    'slope']),
                                                                  ('scaler',
                                                                   StandardScaler(),
                                                                   ['age',
                                                                    'trestbps',
                                                                    'chol',
                                                                    'thalach',
                                                                    'oldpeak',
                                                                    'ca',
                                                                    'thal']),
                                                                  ('as_is',
                                                                   'passthrough',
                                                                   ['sex',
                                                                    'fbs',
                                                                    'exang'])]))])),
                ('classifier', RandomForestClassifier())])

In [46]:
# проверяем
preds = classifier_pipline.predict(X_test_df)
print(classification_report(Y_test_df, preds))

              precision    recall  f1-score   support

           0       0.77      0.80      0.78        25
           1       0.82      0.79      0.81        29

    accuracy                           0.80        54
   macro avg       0.80      0.80      0.80        54
weighted avg       0.80      0.80      0.80        54



In [47]:
# сохраняем модель
import pickle

with open("models/RandomForestClassifier.pickle", "wb") as file:
    pickle.dump(classifier_pipline, file)

In [48]:
# финальный тест
test_data_file_name = 'data/test_data/final_test_data.csv'
test_data = pd.read_csv(test_data_file_name, delimiter=',', decimal='.')
 
preds = classifier_pipline.predict(test_data.loc[0,all_features])
print(classification_report(test_data[target_col], preds))


              precision    recall  f1-score   support

           0       0.87      0.93      0.90        14
           1       0.93      0.88      0.90        16

    accuracy                           0.90        30
   macro avg       0.90      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30

